In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'  # default is ‘last_expr'

%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/Users/siyuyang/Source/repos/GitHub_MSFT/CameraTraps')

In [3]:
import json
import os
from collections import Counter
import io
from random import sample

from tqdm import tqdm
from azure.cosmos.cosmos_client import CosmosClient
from azure.storage.blob import BlockBlobService
from PIL import Image

from visualization import visualization_utils, visualize_megadb
from data_management.annotations import annotation_constants

# Query for data

This notebook demonstrates the workflow to compile desired sequences of images by querying metadata and downloading the images stored in blob storage.

See the programs that Chris prepared here (internal): https://celads.visualstudio.com/CELA%20Data%20Science%20And%20Analytics/_git/DSnA.CameraTrap?path=%2FPython%2Fclientpr.py

`COSMOS_ENDPOINT` and `COSMOS_KEY` need to be environment variables.

In [5]:
# Initialize Cosmos DB client
url = os.environ['COSMOS_ENDPOINT']
key = os.environ['COSMOS_KEY']
client = CosmosClient(url, credential=key)

In [12]:
database = client.get_database_client('camera-trap')
container_datasets = database.get_container_client('datasets')
container_sequences = database.get_container_client('sequences')

## Store the `datasets` table
which records the location and access levels of each dataset.

In [9]:
%%time

query = '''SELECT * FROM datasets d'''

result_iterable = container_datasets.query_items(query=query, enable_cross_partition_query=True)

datasets = {i['dataset_name']:{k: v for k, v in i.items() if not k.startswith('_')} for i in iter(result_iterable)}

print('Length of results:', len(datasets))

Length of results: 18
CPU times: user 6.03 ms, sys: 1.92 ms, total: 7.95 ms
Wall time: 218 ms


## Select image entries

Example: top 1000 images from a given dataset with bounding boxes, selecting the file name and the dataset so we can plot the labels.

In [13]:
%%time

result_iterable = container_sequences.query_items(
    query='''
SELECT TOP @top_n seq
FROM sequences seq JOIN im IN seq.images 
WHERE ARRAY_LENGTH(im.bbox) > 0
''',
    parameters=[
        dict(name='@top_n', value=1000)
    ],
    partition_key='peaceparks_201908_humans'
)

results = [item['seq'] for item in iter(result_iterable)]

print('Length of results:', len(results))

Length of results: 1000
CPU times: user 98.3 ms, sys: 11.2 ms, total: 109 ms
Wall time: 1.7 s


In [14]:
results[-1]

{'images': [{'file': 'Day/1/IMAG0640.JPG',
   'bbox': [{'category': 'person', 'bbox': [0.2349, 0.3966, 0.07641, 0.2099]}],
   'image_id': 'Day~1~IMAG0640'}],
 'location': '1',
 'class': ['human'],
 'dataset': 'peaceparks_201908_humans',
 'seq_id': 'dummy_5c85f601b81d40c08ad592b4aa8ebdc4',
 'datetime': '2019:06:27 14:16:54',
 'id': '229a68a8-746c-4221-bc21-71f529f587c4',
 '_rid': 'WjB+AJ4IhTjqAwAAAAAAAA==',
 '_self': 'dbs/WjB+AA==/colls/WjB+AJ4IhTg=/docs/WjB+AJ4IhTjqAwAAAAAAAA==/',
 '_etag': '"1a0040a4-0000-0500-0000-5dc5ede80000"',
 '_attachments': 'attachments/',
 '_ts': 1573252584}

## Download images and visualize labels

For large batches, download using `multiprocessing.ThreadPool`.

In [ ]:
sample_size = 2
sample_res = sample(results, sample_size)

for seq in sample_res:
    for im in seq['images']:
        if 'bbox' not in im or len(im['bbox']) == 0:
            continue
    
        dataset = seq['dataset']
        storage_account = datasets[dataset]['storage_account']
        storage_container = datasets[dataset]['container']
        storage_sas_key = datasets[dataset]['container_sas_key']
        path_prefix = datasets[dataset]['path_prefix']

        blob_service = visualize_megadb.get_blob_service(datasets, dataset)
        stream = io.BytesIO()
        _ = blob_service.get_blob_to_stream(storage_container, os.path.join(path_prefix, im['file']), stream)
        image = Image.open(stream)

        visualization_utils.render_megadb_bounding_boxes(im['bbox'], image)
        print('from dataset {}'.format(dataset))
        image